<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos - Primavera 2024</strong></center>

### **Cuerpo Docente:**

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliar: Eduardo Moya
- Ayudantes: Nicolás Ojeda, Melanie Peña, Valentina Rojas

### Equipo: SUPER IMPORTANTE - notebooks sin nombre no serán revisados

- Nombre de alumno 1: Tomás Ignacio Reyes Oyarzún


### **Link de repositorio de GitHub:** [Repositorio - TR ](https://github.com/TomiReyes/MDS7202-TR)

### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

# Importamos librerias útiles

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import pickle

from sklearn import set_config
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler, FunctionTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

from xgboost import XGBRegressor

import optuna
from optuna.samplers import TPESampler
from optuna.integration import XGBoostPruningCallback
import optuna.visualization as vis

optuna.logging.set_verbosity(optuna.logging.WARNING)

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Al ver el gran potencial y talento que usted ha demostrado en el campo de la ciencia de datos, Fiu lo contrata como data scientist para que forme parte de su nuevo emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
df = pd.read_csv('sales.csv')
df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


Se eliminan las columnas city, lat y long porque son la misma información que la variable shop_1. Es decir, sabiendo el valor de la variable shop puedo tener es información, por lo que es rebundante. Y se elimina pop, porque no entrega información, no es un identificador ni tampoco significa algo. 

In [3]:
df = df.drop(['city', 'lat', 'long', 'pop'], axis=1)
df.head()

,id,date,shop,brand,container,capacity,price,quantity
0,0,31/01/12,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,31/01/12,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,31/01/12,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,31/01/12,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,31/01/12,shop_1,adult-cola,can,330ml,0.39,25696


## 1 Generando un Baseline (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad. [0.5 puntos]
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas. [1 punto]
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas. `Nota:` Utilice el método `.set_output(transform='pandas')` para obtener un DataFrame como salida del `ColumnTransformer` [1 punto]
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios. [0.5 punto]
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio? [0.5 puntos]
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`? [1 punto]
7. Guarde ambos modelos en un archivo .pkl (uno cada uno) [0.5 puntos]

In [4]:
set_config(transform_output="pandas")

train, temp = train_test_split(df, test_size=0.3, random_state=10)
val, test = train_test_split(temp, test_size=1/3, random_state=10)

In [5]:
def extract_date_parts(df):
    df['day'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.day.astype('category')
    df['month'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.month.astype('category')
    df['year'] = pd.to_datetime(df['date'], format='%d/%m/%y').dt.year.astype('category')
    return df.drop(columns=['date'])

date_transformer = FunctionTransformer(extract_date_parts, validate=False)
train = date_transformer.transform(train)
val = date_transformer.transform(val)
test = date_transformer.transform(test)

In [6]:
categorical_cols = ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numeric_cols = ['price']

preprocessor = ColumnTransformer(
    transformers=[
        ('num', MinMaxScaler(), numeric_cols),
        ('cat', OneHotEncoder(sparse_output=False), categorical_cols) 
    ]
).set_output(transform='pandas')

In [7]:
pipeline_dummy = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

y_train = train['quantity']
X_train = train.drop(columns=['quantity'])

pipeline_dummy.fit(X_train, y_train)

Pipeline(steps=[('preprocessor',
                 ColumnTransformer(transformers=[('num', MinMaxScaler(),
                                                  ['price']),
                                                 ('cat',
                                                  OneHotEncoder(sparse_output=False),
                                                  ['shop', 'brand', 'container',
                                                   'capacity', 'day', 'month',
                                                   'year'])])),
                ('regressor', DummyRegressor())])

In [8]:
val_predictions = pipeline_dummy.predict(val.drop(columns='quantity'))
mae_dummy = mean_absolute_error(val['quantity'], val_predictions)

print(f'MAE con DummyRegressor: {mae_dummy}')

MAE con DummyRegressor: 13591.409320970211


In [9]:
df['quantity'].describe()

count      7456.000000
mean      29408.428380
std       17652.985675
min        2953.000000
25%       16572.750000
50%       25294.500000
75%       37699.000000
max      145287.000000
Name: quantity, dtype: float64

¿Cómo se interpreta esta métrica para el contexto del negocio?

Considerando que son cantidades, hay una diferencia promedio de 13591 cantidad de producto entre la cantidad predicha y la cantidad real. Lo cual indica un error considerable.

In [10]:
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

pipeline_xgb.fit(train.drop(columns='quantity'), train['quantity'])

val_predictions_xgb = pipeline_xgb.predict(val.drop(columns='quantity'))
mae_xgb = mean_absolute_error(val['quantity'], val_predictions_xgb)

print(f'MAE con XGBRegressor: {mae_xgb}')

MAE con XGBRegressor: 2488.454192112309


¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?

El MAE disminuye más de 6 veces respecto al DummyRegressor, por lo que es mucho mejor. En este sentido el error es, en promedio, de 2488 unidades de producto entre lo predicho y lo real.

In [11]:
with open('pipeline_dummy.pkl', 'wb') as f:
    pickle.dump(pipeline_dummy, f)

with open('pipeline_xgb.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 2. Forzando relaciones entre parámetros con XGBoost (10 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline` con `XGBRegressor`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. [6 puntos]

>Hint 1: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

>Hint 2: Puede obtener el nombre de las columnas en el paso anterior al modelo regresor mediante el método `.get_feature_names_out()`

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación. [1 puntos]

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo? [2 puntos]

4. Guarde su modelo en un archivo .pkl [1 punto]

In [12]:
X_train_transformed = preprocessor.fit_transform(X_train)

feature_names = preprocessor.get_feature_names_out()

monotonic_constraints = {"num__price": -1} 

xgb_model = XGBRegressor(monotone_constraints=monotonic_constraints)

In [13]:
pipeline_xgb = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('regressor', xgb_model)
])

In [14]:
pipeline_xgb.fit(train.drop(columns='quantity'), train['quantity'])

val_predictions_xgb = pipeline_xgb.predict(val.drop(columns='quantity'))
mae_xgb = mean_absolute_error(val['quantity'], val_predictions_xgb)

print(f'MAE con XGBRegressor (restricción monotónica): {mae_xgb}')

MAE con XGBRegressor (restricción monotónica): 2445.3252430435477


¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?

Cambia pero no de manera considerable, pasa de 2488 a 2445, por lo que no tiene el impacto esperado, mi amigo no tenía razón. 

In [15]:
with open('pipeline_xgb_monotonic.pkl', 'wb') as f:
    pickle.dump(pipeline_xgb, f)

## 1.3 Optimización de Hiperparámetros con Optuna (20 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se pide que su optimización considere lo siguiente:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)

Para ello se pide los siguientes pasos:
1. Implemente una función `objective()` que permita minimizar el `MAE` en el conjunto de validación. Use el método `.set_user_attr()` para almacenar el mejor pipeline entrenado. [10 puntos]
2. Fije el tiempo de entrenamiento a 5 minutos. [1 punto]
3. Optimizar el modelo y reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
4. Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados? [5 puntos]
5. Guardar su modelo en un archivo .pkl [1 punto]

In [16]:
def objective(trial):
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), ['price']),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency), 
             ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year'])
        ]
    ).set_output(transform='pandas')

    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=10  
    )

    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    pipeline.fit(train.drop(columns='quantity'), train['quantity'])

    val_predictions = pipeline.predict(val.drop(columns='quantity'))
    mae = mean_absolute_error(val['quantity'], val_predictions)

    trial.set_user_attr("best_pipeline", pipeline)

    return mae

In [17]:
study = optuna.create_study(direction='minimize', sampler=TPESampler(seed=10))
study.optimize(objective, timeout=300) 

In [18]:
print(f"Best MAE: {study.best_value}")
print(f"Number of Trials: {len(study.trials)}")
print(f"Best Hyperparameters: {study.best_params}")

best_pipeline = study.best_trial.user_attrs["best_pipeline"]

Best MAE: 1927.4449561955064
Number of Trials: 350
Best Hyperparameters: {'min_frequency': 0.06702381297671205, 'learning_rate': 0.09992302642005202, 'n_estimators': 730, 'max_depth': 8, 'max_leaves': 100, 'min_child_weight': 4, 'reg_alpha': 0.7832290027734599, 'reg_lambda': 0.1613969679288699}


¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

El resultado es un MAE de 1927, el cual es una mejora respecto a la sección anterior, que era 2445. La mejora se debe a la optimización de hiperparámetros que permite obtener una mejor versión del modelo.

Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?

- Hiperparámetros:
    - `learning_rate`: evalúa a que ritmo va aprendiendo el modelo, considerando que hay que evitar el sobreajuste, el rango indicado tiene sentido.
    - `n_estimators`: número de árboles de modo de equilibrar entre tiempo de ejecución y mejor rendimiento. El rango es suficiente para encontrar ese equilibrio.
    - `max_depth`: profundidad de cada árbol. El rango establecido es útil para determinar ese valor evitando sobreajuste. 
    - `max_leaves`: Máximo de hojas de cada árbol, mientras más hojas más se dividen las desiciones por lo que el rango útil permite encontrar ese valor óptimo.
    - `min_child_weight`: Mínimo peso de una hoja, quiere decir cuanta importancia tiene cada una en la desición de cada nodo, el rango establecido hace sentido.
    - `reg_alpha`: Regularización L1, simplifica el modelo, el rango establece eso. 
    - `reg_lambda`: Regularización L2, evita sobreajuste, tomando en cuenta el rango establecido. 
    - `min_frequency`: Mínimo de cantidad para incluir una categoría, de modo de no incluir casos muy excepcionales. el rango establecido permite eso. 

In [19]:
with open('best_pipeline_optuna.pkl', 'wb') as f:
    pickle.dump(best_pipeline, f)

## 4. Optimización de Hiperparámetros con Optuna y Prunners (17 puntos)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento? [2 puntos]
- Redefinir la función `objective()` utilizando `optuna.integration.XGBoostPruningCallback` como método de **Prunning** [10 puntos]
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos [1 punto]
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto? [3 puntos]
- Guardar su modelo en un archivo .pkl [1 punto]

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?

El prunning va eliminando los casos de poco impacto, es decir, los trials que no tienen buenos resultados de modo de ahorrar tiempo y recursos. Detiene el entrenamiento en casos de que no vaya a tener un impacto considerable. Reduce el tiempo de entrenamiento haciendolo más eficiente. 

In [20]:
def objective(trial):
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0.0, 1.0)
    reg_lambda = trial.suggest_float('reg_lambda', 0.0, 1.0)

    preprocessor = ColumnTransformer(
        transformers=[
            ('num', MinMaxScaler(), ['price']),
            ('cat', OneHotEncoder(sparse_output=False, min_frequency=min_frequency),
             ['shop', 'brand', 'container', 'capacity', 'day', 'month', 'year'])
        ]
    ).set_output(transform='pandas')

    X_train_transformed = preprocessor.fit_transform(train.drop(columns='quantity'))
    X_val_transformed = preprocessor.transform(val.drop(columns='quantity'))

    pruning_callback = XGBoostPruningCallback(trial, "validation_0-mae")

    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        random_state=10,
        enable_categorical=True,  
        early_stopping_rounds=10,
        callbacks=[pruning_callback],
        eval_metric='mae'  
    )

    model.fit(
        X_train_transformed, train['quantity'],
        eval_set=[(X_val_transformed, val['quantity'])]
    )

    val_predictions = model.predict(X_val_transformed)
    mae = mean_absolute_error(val['quantity'], val_predictions)

    trial.set_user_attr("best_model", model)

    return mae

study = optuna.create_study(direction='minimize', sampler=optuna.samplers.TPESampler(seed=42))
study.optimize(objective, timeout=300, show_progress_bar=True)


   0%|          | 00:00/05:00

[0]	validation_0-mae:13077.26239
[1]	validation_0-mae:12639.68345
[2]	validation_0-mae:12262.93497
[3]	validation_0-mae:11939.39573
[4]	validation_0-mae:11660.60681
[5]	validation_0-mae:11430.50668
[6]	validation_0-mae:11234.90096
[7]	validation_0-mae:11078.83692
[8]	validation_0-mae:10946.02042
[9]	validation_0-mae:10837.49129
[10]	validation_0-mae:10740.07619
[11]	validation_0-mae:10662.41122
[12]	validation_0-mae:10598.49666
[13]	validation_0-mae:10542.66746
[14]	validation_0-mae:10497.05307
[15]	validation_0-mae:10457.11109
[16]	validation_0-mae:10423.46175
[17]	validation_0-mae:10394.41994
[18]	validation_0-mae:10370.40569
[19]	validation_0-mae:10352.64836
[20]	validation_0-mae:10340.66925
[21]	validation_0-mae:10325.26425
[22]	validation_0-mae:10316.45637
[23]	validation_0-mae:10304.75824
[24]	validation_0-mae:10300.03896
[25]	validation_0-mae:10297.77592
[26]	validation_0-mae:10294.45956
[27]	validation_0-mae:10290.09729
[28]	validation_0-mae:10287.65075
[29]	validation_0-mae:10

In [21]:
print(f"Mejor MAE: {study.best_value}")
print(f"Número de Trials: {len(study.trials)}")
print(f"Mejores Hiperparámetros: {study.best_params}")

best_model_prunning = study.best_trial.user_attrs["best_model"]

Mejor MAE: 2087.1290573846727
Número de Trials: 801
Mejores Hiperparámetros: {'min_frequency': 0.005522117123602399, 'learning_rate': 0.08173068141702859, 'n_estimators': 722, 'max_depth': 8, 'max_leaves': 77, 'min_child_weight': 1, 'reg_alpha': 0.3584657285442726, 'reg_lambda': 0.11586905952512971}


¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?

En este caso el modelo obtiene un peor MAE, dado que aumenta, el número de trial aumenta aunque se deberían elimianr los casos de menor impacto. Algunos hiperparámetros varían harto como el min_frecuency, y algunos no tanto como reg_lamda. Esto puede que afecte el tema de tener menos entrenamiento con el fin de hacerlo más eficiente. De todos modos la diferencia de MAE no es considerable en base a las dimensiones de los datos.

In [22]:
with open("best_model_prunning.pkl", "wb") as f:
    pickle.dump(best_model_prunning, f)

## 5. Visualizaciones (5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización [1 punto]
2. Gráfico de coordenadas paralelas [1 punto]
3. Gráfico de importancia de hiperparámetros [1 punto]

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

In [23]:
fig = vis.plot_optimization_history(study)
fig.show()

fig = vis.plot_parallel_coordinate(study)
fig.show()

fig = vis.plot_param_importances(study)
fig.show()

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados? [0.5 puntos]

Las mejoras notables se observan en los primeros 10 trials. En adelante no tienen gran impacto. 

5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas? [1 punto]

Se observa que hay hiperparámetros que tienen bajo impacto, casi nulo y otros de harta importancia. 

6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo? [0.5 puntos]

El hiperparámetro con mayor impacto es min_frecuency, le sigue reg_alpha, learning_rate y max_leaves. Los otros no tienen mucha importancia

## 6. Síntesis de resultados (3 puntos)

Finalmente:

1. Genere una tabla resumen del MAE en el conjunto de validación obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning. [1 punto]
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento? [0.5 puntos]
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE. [0.5 puntos]
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

In [24]:
X_val_transformed = preprocessor.transform(val.drop(columns='quantity'))
y_val = val['quantity']

In [25]:
X_val_transformed = preprocessor.transform(val.drop(columns='quantity'))
y_val = val['quantity']

with open("best_model_prunning.pkl", 'rb') as f:
    best_model = pickle.load(f)

val_predictions = best_model.predict(X_val_transformed)
mae = mean_absolute_error(y_val, val_predictions)

print(f'MAE prunning: {mae}')


MAE prunning: 2087.1290573846727


In [28]:
X_val = val.drop(columns=['quantity'])
y_val = val['quantity']

model_names = [
    "pipeline_dummy.pkl", "pipeline_xgb.pkl", 
    "pipeline_xgb_monotonic.pkl", "best_pipeline_optuna.pkl", 
]

mae_results = []

for model_name in model_names:
    with open(model_name, 'rb') as f:
        model = pickle.load(f)

    val_predictions = model.predict(X_val)
    mae = mean_absolute_error(y_val, val_predictions)

    mae_results.append((model_name, mae))

df_mae = pd.DataFrame(mae_results, columns=['Modelo', 'MAE'])

df_mae.loc[len(df_mae)] = ['best_model.pkl', 2087.1290573846727]
df_mae

,Modelo,MAE
0,pipeline_dummy.pkl,13591.409321
1,pipeline_xgb.pkl,2488.454192
2,pipeline_xgb_monotonic.pkl,2445.325243
3,best_pipeline_optuna.pkl,1927.444956
4,best_model.pkl,2087.129057


¿qué modelo obtiene el mejor rendimiento?

Solo viendo MAE, el mejor modelo es el best_pipeline_optuna.

In [27]:
X_test = test.drop(columns=['quantity'])
y_test = test['quantity']

with open("best_pipeline_optuna.pkl", 'rb') as f:
    best_pipeline_optuna = pickle.load(f)

test_predictions = best_pipeline_optuna.predict(X_test)

mae_test = mean_absolute_error(y_test, test_predictions)

print(f'MAE en el conjunto de test: {mae_test}')

MAE en el conjunto de test: 2033.7947696961921


4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto? [1 punto]

Existen diferencias porque trata de un set de datos de menor tamaño y puede que sean datos muy distintos con los que se validó. De todos modos la diferencia es cercana a 100 unidades por lo que no es de gran impacto considerando las dimensiones de los datos.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-106f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>